# data kilder
Data for danmark: https://www.statbank.dk/20050 

Data for Norge: https://www.ssb.no/en/statbank/table/03672/tableViewLayout1/ 

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import statsmodels.api as sm

In [ ]:
# Norway data
df = pd.read_excel('data_raw/roykere_no_72_24.xlsx')
df.head(15)

,"05307: Dagligrøykere og av-og-til-røykere (16-79 år), etter kjønn, alder, år og statistikkvariabel",Unnamed: 1,Unnamed: 2,Unnamed: 3
0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,Andel dagligrøykere (prosent)
2,Begge kjønn,I alt,1973.0,42
3,NaN,NaN,1974.0,41
4,NaN,NaN,1975.0,41
5,NaN,NaN,1976.0,39
6,NaN,NaN,1977.0,38
7,NaN,NaN,1978.0,38
8,NaN,NaN,1979.0,37
9,NaN,NaN,1980.0,36


In [15]:
df_cleaned = df.drop(df.columns[[0, 1]], axis=1)
df_cleaned = df_cleaned.drop(df_cleaned.index[0:2])
df_cleaned.head(30)

,Unnamed: 2,Unnamed: 3
2,1973.0,42
3,1974.0,41
4,1975.0,41
5,1976.0,39
6,1977.0,38
7,1978.0,38
8,1979.0,37
9,1980.0,36
10,1981.0,36
11,1982.0,36


In [ ]:
df_eng = pd.read_excel('data_raw/roykere_eng_00_23.xlsx')
df_eng.head(15)

,Unnamed: 0,Unnamed: 1,Unnamed: 2
0,NaN,year,All above 16
1,NaN,2023,10.5
2,NaN,2022,11
3,NaN,2021,12.6
4,NaN,2020,14.4
5,NaN,2019,15.5
6,NaN,2018,16.3
7,NaN,2017,16.4
8,NaN,2016,15.8
9,NaN,2015,17.9


In [19]:
df_eng_cleaned = df_eng.drop(df_eng.columns[[0]], axis=1)
df_eng_cleaned.head(30)

,Unnamed: 1,Unnamed: 2
0,year,All above 16
1,2023,10.5
2,2022,11
3,2021,12.6
4,2020,14.4
5,2019,15.5
6,2018,16.3
7,2017,16.4
8,2016,15.8
9,2015,17.9


In [20]:
df_cleaned.to_excel('data_washed/norway_smokers.xlsx')
df_eng_cleaned.to_excel('data_washed/england_smokers.xlsx')